In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from asbe.base import *
from asbe.models import *
from asbe.estimators import *
from asbe.helper import *
from dataclasses import dataclass
import numpy as np
from collections.abc import Callable
from typing import Union


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution
pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.


In [ ]:
# Data generating process

In [ ]:
@dataclass
class DataGenerator():
    ds: str = None
    dgp_x : Union[Callable, None] = None
    dgp_t : Union[Callable, None] = None
    dgp_y : Union[Callable, None] = None
    
    def get_X(self, method : str = "random", no_query : int = 1):
        if self.ds is not None:
            ix = np.random.randint(low = 0, high = self.ds["X_pool"].shape[0], size=(no_query))
            X_new = self.ds["X_pool"][ix,:]
            self.selected_ix = ix
        else:
            X_new = self.dgp_x()
        self.X_new = X_new
        return X_new
            
    def get_t(self, X_new=None):
        if X_new is None:
            X_new = self.X_new
        if self.ds is not None:
            t_new = self.ds["t_pool"][self.selected_ix]
        else:
            t_new = self.dgp_t(X_new)
        self.t_new = t_new
        return t_new
        
    def get_y(self, X_new=None, t_new=None):
        if X_new is None:
            X_new = self.X_new
        if t_new is None:
            t_new = self.t_new
        if self.ds is not None:
            y_new = np.where(t_new == 1,
                             self.ds["y1_pool"][self.selected_ix],
                             self.ds["y0_pool"][self.selected_ix])
        else:
            y_new = self.dgp_y(X_new, t_new)
        return y_new
            
    def get_data(self):
        X_new = self.get_X()
        t_new = self.get_t()
        y_new = self.get_y()
        return (X_new, t_new, y_new)

In [ ]:
dg = DataGenerator(ds = {"X_pool":np.array([[1,2,3,4,5,6],[7,8,9,0,1,2]]),
                         "t_pool":np.array( [1,0]),
                         "y1_pool":np.array( [2, 10]),
                         "y0_pool":np.array( [1, 7])})

In [ ]:
dg.get_data()

(array([[7, 8, 9, 0, 1, 2]]), array([0]), array([7]))

In [ ]:
IHDP = DataGenerator(ds = get_ihdp_dict(1))

test_data = IHDP.get_data()
assert len(test_data) == 3
assert type(test_data[0]) is np.ndarray
assert test_data[1].shape == (1)

AssertionError: 

In [ ]:
test_data[1].shape